In [6]:
import pandas as pd
import numpy as np
from googlesearch import search
from bs4 import BeautifulSoup
import requests
import time
import os

In [9]:
def get_info(query):
    
    imdb = None
    
    while True:
        try:
            for url in search(query, stop=5, pause=2, only_standard=True):
                if 'www.imdb.com' in url:
                    imdb = url
                    break        
            if imdb:
                source = requests.get(imdb)
                soup = BeautifulSoup(source.text, 'lxml')
                break
            else:
                return 
        except:    
            time.sleep(20)
            continue
        
    
    lang = soup.find('h4', text='Language:')
    if lang:
        lang = lang.findNext('a').text

    director = soup.find('h4', text='Director:')
    if director:
        director = director.findNext('a').text
    else:
        director = soup.find('h4', text='Directors:')
        if director:
            director = director.parent.text
            director = director.split(':')[-1].strip()

    cast = soup.find('h4', text='Stars:')
    if cast:
        cast = cast.parent.text
        cast = cast.split('|')[0].split(':')[-1].strip()

    date = soup.find(title="See more release dates")
    if date:
        date = date.text.strip()

    country = soup.find('h4', text='Country:')
    if country:
        country = country.parent.text
        country = ', '.join([a.strip() for a in country.split(':')[-1].split('|')])

    runtime = soup.find('time')
    if runtime:
        runtime = runtime.text.strip()
    
    return [lang, director, cast, date, country, runtime]

In [16]:
df = pd.read_excel("C:\Dataframes\EIDR.xlsx")
for i in range(38000, 38145):
    print("Progress: " + str(i))
    try:
        info = get_info(df['Query'][i])
        df['Original Language'][i] = info[0]
        df['Director'][i] = info[1]
        df['Actors'][i] = info[2]
        df['Release Date'][i] = info[3]
        df['Country Of Origin'][i] = info[4]
        df['Running Time'][i] = info[5]
    except:
        continue

df.to_excel("C:\Dataframes\EIDR.xlsx", index=False)

Progress: 38000
Progress: 38001
Progress: 38002
Progress: 38003
Progress: 38004
Progress: 38005
Progress: 38006
Progress: 38007
Progress: 38008
Progress: 38009
Progress: 38010
Progress: 38011
Progress: 38012
Progress: 38013
Progress: 38014
Progress: 38015
Progress: 38016
Progress: 38017
Progress: 38018
Progress: 38019
Progress: 38020
Progress: 38021
Progress: 38022
Progress: 38023
Progress: 38024
Progress: 38025
Progress: 38026
Progress: 38027
Progress: 38028
Progress: 38029
Progress: 38030
Progress: 38031
Progress: 38032
Progress: 38033
Progress: 38034
Progress: 38035
Progress: 38036
Progress: 38037
Progress: 38038
Progress: 38039
Progress: 38040
Progress: 38041
Progress: 38042
Progress: 38043
Progress: 38044
Progress: 38045
Progress: 38046
Progress: 38047
Progress: 38048
Progress: 38049
Progress: 38050
Progress: 38051
Progress: 38052
Progress: 38053
Progress: 38054
Progress: 38055
Progress: 38056
Progress: 38057
Progress: 38058
Progress: 38059
Progress: 38060
Progress: 38061
Progress